In [1]:
import pandas as pd
import numpy as np
import pickle
from typing import Optional, Union, Tuple
from cmapPy.pandasGEXpress.parse import parse
import numpy as np
import json
from collections import Counter
import sys
from numpy.core.multiarray import ndarray
import sys
import csv 
import requests 
import xml.etree.ElementTree as ET 
from xml.dom import minidom

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Parsing the XML file

## Importing drugbank dataset

In [56]:
drugbank = pd.read_csv("drugbank.csv",low_memory=False)
drugbank = drugbank.dropna(how='all', axis='columns')
drugbank.head(2)

atc-level1 atc-level2 atc-level3 atc-level4 atc-level5 inchi inchi_key  \
0          B        B01       B01A      B01AE    B01AE02   NaN       NaN   
1          L        L01       L01X      L01XC    L01XC06   NaN       NaN   

        name     pubchem smiles ...  syn59 syn6 syn60 syn61 syn62 syn63 syn64  \
0  Lepirudin  46507011.0    NaN ...    NaN  NaN   NaN   NaN   NaN   NaN   NaN   
1  Cetuximab  46507042.0    NaN ...    NaN  NaN   NaN   NaN   NaN   NaN   NaN   

  syn7 syn8 syn9  
0  NaN  NaN  NaN  
1  NaN  NaN  NaN  

[2 rows x 74 columns]

In [57]:
p = pd.read_csv("../Data/GSE70138_Broad_LINCS_pert_info.txt",delimiter="\t")
p.canonical_smiles = p.canonical_smiles.str.upper()
p.head(2)
p.to_csv("metadata.csv",index=False)

In [4]:
X = pd.read_pickle("../Data/full")

In [5]:
# two different smiles for the same inchi_key
print(drugbank[drugbank.inchi_key=='QHMWCHQXCUNUAK-UHFFFAOYSA-N'].smiles.tolist()[0])
print(p[p.inchi_key=='QHMWCHQXCUNUAK-UHFFFAOYSA-N'].canonical_smiles.tolist()[0])

FC1=CC=C(C=C1)C(N1C=CN=C1)(C1=CC=CC=C1)C1=CC=CC=C1F
FC1CCC(CC1)C(C1CCCCC1)(C1CCCCC1F)N1CCNC1


## Get the atc codes via drugbank

In [48]:
def check_smile(smile1, smile2):
    smile_list1 = sorted([string for string in smile1.replace('=', '').replace(')', ' ').replace('(', ' ').split(' ') if string!=''])
    smile_list2 = sorted([string for string in smile2.replace('=', '').replace(')', ' ').replace('(', ' ').split(' ') if string!=''])
    
    return smile_list1 == smile_list2

In [58]:
li=[]
tar = np.unique(X.target)
unseen=0
for i in range(len(tar)):
    ret = dict()
    if(len(p[p.pert_id==tar[i]])==0):
        unseen+=1
        continue
    inchi_key = p[p.pert_id==tar[i]].inchi_key.tolist()[0]
    smiles = p[p.pert_id==tar[i]].canonical_smiles.tolist()[0]
    ret["inchi_key"]=inchi_key
    ret["smiles"]=smiles
#     print(inchi_key)
#     print(inchi_key, smiles)
    try:
        for l in range(1,6):
            if(inchi_key):
                ret["level"+str(l)] = drugbank[drugbank.inchi_key==inchi_key]["atc-level"+str(l)].tolist()[0]
#             if(smiles):
#                 print(smiles)
#                 ret["level"+str(l)] = drugbank[[check_smile(smile, smiles) for smile in drugbank.smiles]]["atc-level"+str(l)].tolist()[0]
        ret["name"] = drugbank[drugbank.inchi_key==inchi_key]["name"].tolist()[0]
    except:
        pass
    sys.stdout.write('\r%d/%d' % (i, len(tar)))
    li.append(ret)
    
print("\nUnseen perturbagen id's in metadata: ",unseen)

2169/2170
Unseen perturbagen id's in metadata:  0


In [62]:
# Instance of dictionary
li[0]

{'inchi_key': 'IETKPTYAGKZLKY-UHFFFAOYSA-N',
 'level1': nan,
 'level2': nan,
 'level3': nan,
 'level4': nan,
 'level5': nan,
 'name': 'Balaglitazone',
 'smiles': 'CN1C(COC2CCC(CC3SC(=O)NC3=O)CC2)NC2CCCCC2C1=O'}

In [63]:
len(li)

2170

## Import the final dataset

In [94]:
# pd.DataFrame(li).to_csv("atc-codes.csv",index=False)
atc = pd.read_csv("atc-codes.csv")
atc.name=atc.name.str.lower()
atc.head(2)

inchi_key level1 level2 level3 level4   level5  \
0  IETKPTYAGKZLKY-UHFFFAOYSA-N    NaN    NaN    NaN    NaN      NaN   
1  QHMWCHQXCUNUAK-UHFFFAOYSA-N      G    G01   G01A  G01AF  G01AF18   

            name                                         smiles  
0  balaglitazone  CN1C(COC2CCC(CC3SC(=O)NC3=O)CC2)NC2CCCCC2C1=O  
1   flutrimazole       FC1CCC(CC1)C(C1CCCCC1)(C1CCCCC1F)N1CCNC1

In [82]:
print(atc.shape)
atc.isnull().sum()

(2170, 9)


inchi_key       0
level1       1484
level2       1484
level3       1484
level4       1484
level5       1484
name         1069
smiles          0
pert_id      1069
dtype: int64

## Import original ATC standalone dataset

In [70]:
atc_data = pd.read_excel("atc.xlsx")
atc_data.head(2)

Class ID Preferred Label Synonyms  \
0  http://purl.bioontology.org/ontology/UATC/A03AX13       silicones      NaN   
1  http://purl.bioontology.org/ontology/UATC/J01DB07     cefatrizine      NaN   

   Definitions  Obsolete       CUI  \
0          NaN     False  C0037114   
1          NaN     False  C0007545   

                                      Semantic Types  \
0  http://purl.bioontology.org/ontology/STY/T109|...   
1  http://purl.bioontology.org/ontology/STY/T195|...   

                                           Parents  ATC LEVEL Is Drug Class  \
0  http://purl.bioontology.org/ontology/UATC/A03AX        5.0           NaN   
1  http://purl.bioontology.org/ontology/UATC/J01DB        5.0           NaN   

                         Semantic type UMLS property  
0  http://purl.bioontology.org/ontology/STY/T109|...  
1  http://purl.bioontology.org/ontology/STY/T195|...

In [73]:
atc_data = atc_data[['Preferred Label','Class ID']]
atc_data.rename(columns={'Preferred Label':"name",'Class ID':'atc' },inplace=True)

In [78]:
def make_level(c):
    atc_class = c.split('/')[-1]
    return atc_class

make_level("http://purl.bioontology.org/ontology/UATC/A03AX13")

'A03AX13'

In [83]:
atc_data.atc = atc_data.atc.apply(make_level)
atc_data.to_csv("Final/atc.csv",index=False)
atc_data.head(2)

name      atc
0    silicones  A03AX13
1  cefatrizine  J01DB07

## Get pert_iname from pert_id

In [88]:
id_to_name = dict()
name_to_id = dict()
tar = np.unique(X.target)
count = 0
for i in range(len(tar)):
    a = p[p.pert_id==tar[i]].pert_iname
    if(a.to_string().split()[1]==''):
        a = a.to_string().split()[0]
    else:
        a = a.to_string().split()[1]
    a=a.translate({ord(')'): None})
    a=a.translate({ord(']'): None})
#     sys.stdout.write('\r%d/%d\t\t%s' % (i, len(tar),a))
#     print('%d/%d\t\t%s' % (i, len(tar),a))
    id_to_name[tar[i]]=a
    try:
        name_to_id[a]
        name_to_id[a]=name_to_id[a]+[tar[i]]
        count+=1
#         print(a,name_to_id[a])
    except:
        name_to_id[a]=[tar[i]]
#         print(a,name_to_id[a])

### Creating 2 different dictionaries with name_to_id

In [87]:
name_to_id_single = dict()
name_to_id_multiple = dict()
for i in name_to_id.keys():
    if(len(name_to_id[i])>1):
        name_to_id_multiple[i]=name_to_id[i]
    elif(len(name_to_id[i])==1):
        name_to_id_single[i]=name_to_id[i]
        
len(name_to_id_multiple),len(name_to_id_single), len(name_to_id_multiple)+len(name_to_id_single)

(82, 1744, 1826)

In [89]:
print(len(name_to_id))
len(id_to_name)

1826


2170

## Get ATC codes from OG data

In [98]:
atc = atc[['name','level5']]
atc.rename(columns={"level5":"atc"},inplace=True)

In [101]:
meta  = pd.read_csv("Final/meta.csv")

In [107]:
print(atc.shape)
atc.head(2)

(2170, 2)


name      atc
0  balaglitazone      NaN
1   flutrimazole  G01AF18

In [114]:
len(np.unique(list(atc.name)))

1101

In [108]:
print(meta.shape)
meta.head(2)

(2170, 4)


smiles  \
0                CCN(CC)CCCCN1C2CCCCC2OC2CCC(CL)CC12   
1  CCCCCCCCCCCCCC(=O)O[C@@H]1[C@@H](C)[C@]2(O)[C@...   

                        name             id                    inchi_key  
0                    10-DEBC  BRD-K70792160  GYBXAGDWMCJZJK-UHFFFAOYSA-N  
1  phorbol-myristate-acetate  BRD-K68552125  PHEDXBVPIONUQT-RGYGYFBISA-N

In [116]:
pd.merge(atc_data,meta,left_on='name',right_on='name').shape

(1000, 5)

In [61]:
# 1 Instance of dict
len(li1)

768

In [13]:
# atc = atc.drop(['Unnamed: 0'],axis=1)
atc['pert_id'] = atc.name
atc.pert_id = atc.pert_id.replace(name_to_id) 
atc.isnull().sum()
atc.head(2)

inchi_key level1 level2 level3 level4   level5  \
0  IETKPTYAGKZLKY-UHFFFAOYSA-N    NaN    NaN    NaN    NaN      NaN   
1  QHMWCHQXCUNUAK-UHFFFAOYSA-N      G    G01   G01A  G01AF  G01AF18   

            name                                         smiles        pert_id  
0  balaglitazone  CN1C(COC2CCC(CC3SC(=O)NC3=O)CC2)NC2CCCCC2C1=O  BRD-A00147595  
1   flutrimazole       FC1CCC(CC1)C(C1CCCCC1)(C1CCCCC1F)N1CCNC1  BRD-A00218260

In [14]:
newatc = pd.DataFrame(li1)
newatc.isnull().sum()

inchi_key    0
level1       0
level2       0
level3       0
level4       0
level5       0
name         0
pert_id      0
smiles       0
dtype: int64

# Append Data

In [15]:
wholeatc = newatc.append(atc)
wholeatc.isnull().sum()

inchi_key       0
level1       1484
level2       1484
level3       1484
level4       1484
level5       1484
name         1069
pert_id      1069
smiles          0
dtype: int64

In [16]:
for i in Counter(wholeatc.name):
    if(Counter(wholeatc.name)[i]>1):
        print(i,Counter(wholeatc.name)[i])
len(wholeatc.name)

flutrimazole 2
biperiden 2
bupivacaine 2
nitrendipine 2
meptazinol 2
lofexidine 2
alizapride 2
mepivacaine 2
metoprolol 2
chlorphenamine 2
bupropion 2
esmolol 2
labetalol 2
bopindolol 2
racecadotril 2
flecainide 2
verapamil 2
propranolol 2
carvedilol 2
lorazepam 2
pomalidomide 2
troglitazone 2
doxazosin 2
alaproclate 2
milnacipran 2
rebamipide 2
indobufen 2
butoconazole 2
bambuterol 2
ibuprofen 2
lenalidomide 2
phensuximide 2
trimipramine 2
perhexiline 2
mianserin 2
ondansetron 2
mecamylamine 2
chlormezanone 2
praziquantel 2
amlodipine 2
benfluorex 2
pantoprazole 2
itraconazole 2
ifenprodil 2
ofloxacin 2
warfarin 2
aminoglutethimide 2
everolimus 2
propafenone 2
nicardipine 2
tolperisone 2
acebutolol 2
bicalutamide 2
disopyramide 2
felodipine 2
fluoxetine 2
methocarbamol 2
tioconazole 2
sotalol 2
etoposide 2
fenticonazole 2
metixene 2
suprofen 2
ibrutinib 2
rabeprazole 2
ketorolac 2
gemifloxacin 2
ornidazole 2
pinacidil 2
indoprofen 2
pentobarbital 2
proglumide 2
doxapram 2
citalopram 2

2938

In [17]:
len(atc.name) + len(newatc.name)

2938

In [18]:
print(atc[atc.name=='saracatinib'])
print('\n',newatc[newatc.name=='saracatinib'],'\n')
print(wholeatc[wholeatc.name=='saracatinib'])

                        inchi_key level1 level2 level3 level4 level5  \
651   OUKYUETWWIPKQR-UHFFFAOYSA-N    NaN    NaN    NaN    NaN    NaN   
1786  OUKYUETWWIPKQR-UHFFFAOYSA-N    NaN    NaN    NaN    NaN    NaN   

             name                                             smiles  \
651   saracatinib  CN1CCN(CCOC2CC(OC3CCOCC3)C3C(NC4C5OCOC5CCC4CL)...   
1786  saracatinib  CN1CCN(CC1)CCOC2=CC(=C3C(=C2)N=CN=C3NC4=C(C=CC...   

            pert_id  
651   BRD-U07805514  
1786  BRD-U07805514  

 Empty DataFrame
Columns: [inchi_key, level1, level2, level3, level4, level5, name, pert_id, smiles]
Index: [] 

                        inchi_key level1 level2 level3 level4 level5  \
651   OUKYUETWWIPKQR-UHFFFAOYSA-N    NaN    NaN    NaN    NaN    NaN   
1786  OUKYUETWWIPKQR-UHFFFAOYSA-N    NaN    NaN    NaN    NaN    NaN   

             name        pert_id  \
651   saracatinib  BRD-U07805514   
1786  saracatinib  BRD-U07805514   

                                                 smiles  
65

In [19]:
def common_member(a, b): 
    a_set = set(a) 
    b_set = set(b) 
    if (a_set & b_set): 
        print("Number of common elements: ",len(a_set & b_set))
        print("Total perts: ",len(set(a_set|b_set)))
        # returns all the perts in 2 sets
        return(set(a_set|b_set))
    else: 
        print("No common elements")  

all = list(common_member(atc.name.str.lower(),newatc.name))
all

Number of common elements:  605
Total perts:  1264


[nan,
 'tetrahydropalmatine',
 'ketoprofen',
 'imexon',
 'cinoxacin',
 'amineptine',
 'refametinib',
 'deoxycholic acid',
 'foretinib',
 'darifenacin',
 'flutrimazole',
 'antipyrine',
 'telmisartan',
 'glycopyrronium',
 'rosuvastatin',
 'mazindol',
 'fiacitabine',
 'metformin',
 'clinafloxacin',
 'adenosine',
 'vadimezan',
 'alagebrium',
 'bemegride',
 'carisoprodol',
 'tenidap',
 'meclizine',
 'naphazoline',
 'sildenafil',
 'fluoxetine',
 '5-(2-phenylpyrazolo[1,5-a]pyridin-3-yl)-1h-pyrazolo[3,4-c]pyridazin-3-amine',
 'pilocarpine',
 'indacaterol',
 'sulfisoxazole',
 'bortezomib',
 'thiocolchicoside',
 'emapunil',
 'dolasetron',
 'iguratimod',
 'novobiocin',
 'indoximod',
 'nizofenone',
 'bupropion',
 'tg100-115',
 'alisertib',
 'nafamostat',
 'fluorometholone',
 'acipimox',
 'alpelisib',
 'pranlukast',
 'chlorprothixene',
 'piroxicam',
 'hymecromone',
 'zileuton',
 'diethylcarbamazine',
 'iowh-032',
 'lamivudine',
 'palonosetron',
 'raloxifene',
 'gm6001',
 'linagliptin',
 'oxfendazol

In [20]:
wholeatc.name = wholeatc.name.str.lower()

In [21]:
wholeatc.to_csv("wholeatc.csv",index=False)

In [22]:
name_to_id["balaglitazone"]

'BRD-A00147595'

In [23]:
wholeatc.isnull().sum()

inchi_key       0
level1       1484
level2       1484
level3       1484
level4       1484
level5       1484
name         1069
pert_id      1069
smiles          0
dtype: int64

In [33]:
df = pd.DataFrame()
n = list(Counter(wholeatc.name).keys())
for i in list(n):
    print(i, end=' ')
    tempdf = wholeatc[(wholeatc.name==i)]
    if len(tempdf)>1:
        test = wholeatc[(wholeatc.name==i) & (wholeatc.smiles!='')]
        print(">1 ",len(test))
        df = df.append(test)
    else:
        print("1 ",len(tempdf))
        df = df.append(tempdf)

flutrimazole >1  1
biperiden >1  1
drospirenone 1  1
bupivacaine >1  1
nitrendipine >1  1
meptazinol >1  1
lofexidine >1  1
alizapride >1  1
mepivacaine >1  1
fulvestrant 1  1
metoprolol >1  1
maraviroc 1  1
chlorphenamine >1  1
bupropion >1  1
docetaxel 1  1
esmolol >1  1
labetalol >1  1
bopindolol >1  1
racecadotril >1  1
doxycycline 1  1
flecainide >1  1
verapamil >1  1
argatroban 1  1
tipranavir 1  1
propranolol >1  1
granisetron 1  1
carvedilol >1  1
pravastatin 1  1
lorazepam >1  1
cefepime 1  1
cisapride 1  1
pomalidomide >1  1
troglitazone >1  1
fluorometholone 1  1
doxazosin >1  1
alaproclate >1  1
milnacipran >1  1
rebamipide >1  1
bucladesine 1  1
indobufen >1  1
butoconazole >1  1
protirelin 1  1
bambuterol >1  1
ibuprofen >1  1
lenalidomide >1  1
phensuximide >1  1
tazobactam 1  1
estriol 1  1
estradiol 1  1
trimipramine >1  1
perhexiline >1  1
mianserin >1  1
ondansetron >1  1
atovaquone 1  1
mecamylamine >1  1
medrysone 1  1
chlormezanone >1  1
praziquantel >1  1
amlodip

dexfenfluramine >1  1
chlorprothixene 1  1
naproxen >1  1
propentofylline >1  1
voglibose 1  1
linopirdine >1  1
tedizolid >1  1
maribavir >1  1
quinidine >1  1
pasireotide >1  1
irbesartan >1  1
tiagabine >1  1
paricalcitol >1  1
idelalisib >1  1
loperamide >1  1
romidepsin >1  1
thiomersal 1  1
emtricitabine >1  1
anagrelide >1  1
dapsone >1  1
venetoclax >1  1
guanethidine >1  1
triptorelin >1  1
gabapentin >1  1
lidoflazine >1  1
epinephrine >1  1
orlistat >1  1
methoxsalen >1  1
erythromycin >1  1
triflupromazine >1  1
temoporfin 1  1
acipimox >1  1
raloxifene >1  1
molsidomine 1  1
gefitinib >1  1
dienogest 1  1
tretinoin 1  1
aprepitant 1  1
avanafil >1  1
nalbuphine >1  1
pazufloxacin >1  1
moxifloxacin >1  1
linezolid >1  1
lodoxamide >1  1
cilostazol >1  1
altretamine >1  1
milrinone >1  1
apixaban >1  1
acetylcysteine 1  1
picotamide >1  1
haloperidol >1  1
lisinopril >1  1
terbinafine >1  1
etizolam >1  1
lacosamide >1  1
amsacrine >1  1
quetiapine >1  1
sertindole >1  1
di

tak-580 1  1
defactinib 1  1
genistein 1  1
camostat 1  1
phenoxymethylpenicillin 1  1
dexniguldipine 1  1
2-methoxyestradiol 1  1
taselisib 1  1
tauroursodeoxycholic acid 1  1
betrixaban 1  1
cc-401 1  1
emapunil 1  1
preladenant 1  1
varlitinib 1  1
zofenopril 1  1
umeclidinium 1  1
antipyrine 1  1
piclamilast 1  1
podofilox 1  1
balapiravir 1  1
teneligliptin 1  1
tecastemizole 1  1
tanzisertib 1  1
methyldopa 1  1
salsalate 1  1
laropiprant 1  1
gsk-2636771 1  1
pilaralisib 1  1
arotinoid acid 1  1
pd-0325901 1  1
pf-04447943 1  1
tolfenamic acid 1  1
canertinib 1  1
meclofenamic acid 1  1
taladegib 1  1
e-2012 1  1
sulfisoxazole 1  1
tirasemtiv 1  1
sarcosine 1  1
siramesine 1  1
gm6001 1  1
hydroxyurea 1  1
biib021 1  1
(4-amino-2-{[1-(methylsulfonyl)piperidin-4-yl]amino}pyrimidin-5-yl)(2,3-difluoro-6-methoxyphenyl)methanone 1  1
tacedinaline 1  1
pictilisib 1  1
ozagrel 1  1
cts-1027 1  1
mk-5108 1  1
ripasudil 1  1
trehalose 1  1
mk-1775 1  1
methimazole 1  1
turofexorate isopr

In [34]:
df.shape

(1264, 9)

In [35]:
cols = list(df)
cols.remove("smiles")
cols.remove("inchi_key")
df1 = df.dropna(how='any')
df1 = df1[cols]

In [36]:
df1.shape

(849, 7)

In [29]:
df.to_csv("cleaned-atc.csv",index=False)

In [117]:
pd.read_csv("cleaned-atc.csv").head(2)

level1 level2 level3 level4   level5          name        pert_id
0      G    G01   G01A  G01AF  G01AF18  flutrimazole  BRD-A00218260
1      N    N04   N04A  N04AA  N04AA02     biperiden  BRD-A00546892

In [118]:
name_to_id_multiple

{'ACLY': ['BRDN0000734492',
  'BRDN0000734551',
  'BRDN0000734605',
  'BRDN0000735220',
  'BRDN0000735469',
  'BRDN0000735471'],
 'AKT1': ['BRDN0001054755',
  'BRDN0001054783',
  'BRDN0001054802',
  'BRDN0001054816',
  'BRDN0001054908',
  'BRDN0001054985',
  'BRDN0001055115'],
 'AKT2': ['BRDN0001054766',
  'BRDN0001054785',
  'BRDN0001054826',
  'BRDN0001054865',
  'BRDN0001054911',
  'BRDN0001054983',
  'BRDN0001055131'],
 'AURKB': ['BRDN0001054786',
  'BRDN0001054845',
  'BRDN0001054886',
  'BRDN0001054938',
  'BRDN0001054946',
  'BRDN0001054962',
  'BRDN0001054968'],
 'AXL': ['BRDN0000579778',
  'BRDN0000987765',
  'BRDN0000987775',
  'BRDN0000987887',
  'BRDN0000987972',
  'BRDN0000988142'],
 'AZD-1480': ['BRD-A29009894', 'BRD-K65928735'],
 'AZD-7762': ['BRD-K46056750', 'BRD-K86525559', 'BRD-U86686840'],
 'BCL2L1': ['BRDN0000563763',
  'BRDN0000733535',
  'BRDN0000733624',
  'BRDN0000733677',
  'BRDN0000733763',
  'BRDN0000733798'],
 'BI-2536': ['BRD-K64890080', 'BRD-U70626184'],
 

In [32]:
drugbank.isnull().sum()
drugbank.shape

(13339, 74)